In [ ]:
# https://www.statistichesulcalcio.com/confronto_squadre_u.php

In [ ]:
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

from models import *
from logger import get_logger
from datetime import datetime

import urllib.request
import re
import time
import sys
import pandas as pd

In [ ]:
logger = get_logger('StatistichePartite')

In [3]:
url = 'https://www.statistichesulcalcio.com/confronto_squadre_u.php'
evento = 'U2.5'

In [4]:
now = (datetime.now()).strftime('%Y-%m-%d')

current_ts_run = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [5]:
logger.info("+-----------------------------+")
logger.info("| STARTING StatistichePartite |")
logger.info("+-----------------------------+")

[2021-02-18 23:31:51,972]:[StatistichePartite]:[<module>]:[INFO]: +-----------------------------+
[2021-02-18 23:31:51,976]:[StatistichePartite]:[<module>]:[INFO]: | STARTING StatistichePartite |
[2021-02-18 23:31:51,978]:[StatistichePartite]:[<module>]:[INFO]: +-----------------------------+


In [6]:
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--window-size=1920,1080")

In [7]:
try:
    logger.info('Avvio Selenium Chrome')
    driver = webdriver.Chrome(executable_path='./chromedriver', chrome_options=chrome_options)

except Exception as e:
    
    logger.error(e)
    raise

[2021-02-18 23:31:53,558]:[StatistichePartite]:[<module>]:[INFO]: Avvio Selenium Chrome
/opt/jupyterhub/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: use options instead of chrome_options
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
logger.info('Navigo %s' % (url,))
driver.get(url)

[2021-02-18 23:31:55,771]:[StatistichePartite]:[<module>]:[INFO]: Navigo https://www.statistichesulcalcio.com/confronto_squadre_u.php


In [9]:
try:
    logger.info('Estraggo la tabella delle statistiche')

    table_node = driver.find_element_by_id('modificante')
    table = table_node.get_attribute('innerHTML')
    table_soup = BeautifulSoup(table, 'lxml')
except Exception as e:
    
    logger.error(e)
    

[2021-02-18 23:31:58,682]:[StatistichePartite]:[<module>]:[INFO]: Estraggo la tabella delle statistiche


In [10]:
try:

    logger.info('Parsing della tabella delle statistiche')
    stat = (pd.read_html(table)[0])
    stat = stat[stat['POS'].str.match(r'^[0-9]+$')]
    stat = stat[['CAMPIONATO', 'SQUADRA', '%', 'AGGIOR']]

    stat.columns = ['campionato', 'squadra', 'performance', 'data_aggiornamento']

    stat['evento'] = evento
    stat['data_run'] = current_ts_run

except Exception as e:
    
    logger.error(e)
    

[2021-02-18 23:31:59,150]:[StatistichePartite]:[<module>]:[INFO]: Parsing della tabella delle statistiche


In [11]:
logger.info('Inserisco le statistiche in tabella')

for ix, item in stat.to_dict('index').items():
    
    try:
        
        Statistiche.create(**item)
        
    except Exception as e:
        
        logger.error(e)
        
        continue
        

[2021-02-18 23:32:00,295]:[StatistichePartite]:[<module>]:[INFO]: Inserisco le statistiche in tabella
/opt/jupyterhub/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1265, "Data truncated for column 'performance' at row 1")
  result = self._query(query)


In [12]:
dettaglio_list = [(x.get_attribute('href'), x.get_attribute('innerHTML')) 
                  for x in table_node.find_elements_by_tag_name('a') 
                  if x.get_attribute('href').find('dettaglio_partite') > 0]

In [13]:
driver.get(dettaglio_list[0][0])

In [14]:
dettaglio_table = driver.find_elements_by_class_name('table-responsive')

In [29]:
dettaglio_df = pd.read_html(dettaglio_table[1].get_attribute('innerHTML'))[0]

In [32]:
dettaglio_df.head()

,0,1,2,3,4
0,30-11-2020,Premier League,Taraz - K. Almaty,1 - 0,(0 - 0)
1,27-11-2020,Premier League,Tobol - Taraz,2 - 0,(1 - 0)
2,24-11-2020,Premier League,Taraz - Kaspij Aktau,0 - 1,(0 - 1)
3,21-11-2020,Premier League,Zhetysu Taldykorgan - Taraz,0 - 1,(0 - 1)
4,08-11-2020,Premier League,Taraz - Shakhtar Karagandy,2 - 4,(0 - 3)


In [35]:
for i in enumerate([1,2,3]):
    
    print(i)

(0, 1)
(1, 2)
(2, 3)


In [33]:
dettaglio_df[2].apply(lambda x: x.split(' - ')).to_list()

[['Taraz', 'K. Almaty'],
 ['Tobol', 'Taraz'],
 ['Taraz', 'Kaspij Aktau'],
 ['Zhetysu Taldykorgan', 'Taraz'],
 ['Taraz', 'Shakhtar Karagandy'],
 ['Taraz', 'FC Astana'],
 ['Okzhetpes', 'Taraz'],
 ['Kyzylzhar', 'Taraz'],
 ['Taraz', 'Kaisar Kyzylorda'],
 ['Ordabasy', 'Taraz'],
 ['Taraz', 'Tobol'],
 ['Kaspij Aktau', 'Taraz'],
 ['Taraz', 'Zhetysu Taldykorgan'],
 ['Shakhtar Karagandy', 'Taraz'],
 ['FC Astana', 'Taraz'],
 ['Taraz', 'Okzhetpes'],
 ['Taraz', 'Kyzylzhar'],
 ['Kaisar Kyzylorda', 'Taraz'],
 ['Taraz', 'Ordabasy'],
 ['K. Almaty', 'Taraz']]

In [28]:
dettaglio_df

0     [1, 0]
1     [2, 0]
2     [0, 1]
3     [0, 1]
4     [2, 4]
5     [1, 0]
6     [0, 0]
7     [1, 0]
8     [2, 2]
9     [1, 1]
10    [0, 0]
11    [0, 2]
12    [0, 2]
13    [1, 1]
14    [1, 1]
15    [2, 3]
16    [0, 0]
17    [1, 1]
18    [3, 2]
19    [2, 1]
Name: 3, dtype: object

In [ ]:
for item in dettaglio_list:
    
    
    
    
    

In [90]:
driver.close()